In [1]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import matplotlib as mpl
import scipy.stats as st
from scipy.stats import (vonmises_line, norm, t, cauchy, lognorm, fisk, skewnorm, exponweib, genlogistic, logistic, laplace)

import random

import glob
import os
from tqdm import tqdm

In [2]:
# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

In [3]:
# Flux Cutoff in Maxwells
fCutoff = 3e21

KPVT = pd.read_csv('data/output/BARD_1976-1993_KPVT_tilts.csv')
KPVT = KPVT.loc[KPVT['BMRFlux']>=fCutoff,:]
KPVT['Date'] = pd.to_datetime(dict(year=KPVT.Year, month=KPVT.Month, day=KPVT.Day, hour=KPVT.Hour, minute=KPVT.Minute))
KPVT['Instrument'] = 'KPVT'

SPMG = pd.read_csv('data/output/BARD_1992-1999_SPMG_tilts.csv')
SPMG = SPMG.loc[SPMG['BMRFlux']>=fCutoff,:]
SPMG['Date'] = pd.to_datetime(dict(year=SPMG.Year, month=SPMG.Month, day=SPMG.Day, hour=SPMG.Hour, minute=SPMG.Minute))
SPMG['Instrument'] = 'SPMG'
SPMG['BMRLabel'] = SPMG['BMRLabel'] + np.max(KPVT['BMRLabel'])

MDIcalibration = 0.8
MDI = pd.read_csv('data/output/BARD_1996-2010_MDI_tilts.csv')
MDI.loc[:, ['PFlux', 'NFlux', 'BMRFlux']] *= MDIcalibration
MDI = MDI.loc[MDI['BMRFlux']>=fCutoff,:]
MDI['Date'] = pd.to_datetime(dict(year=MDI.Year, month=MDI.Month, day=MDI.Day, hour=MDI.Hour, minute=MDI.Minute))
MDI['Instrument'] = 'MDI'
MDI['BMRLabel'] = MDI['BMRLabel'] + np.max(SPMG['BMRLabel'])

HMI = pd.read_csv('data/output/BARD_2010-2016_HMI_tilts.csv')
HMI = HMI.loc[HMI['BMRFlux']>=fCutoff,:]
HMI['Date'] = pd.to_datetime(dict(year=HMI.Year, month=HMI.Month, day=HMI.Day, hour=HMI.Hour, minute=HMI.Minute))
HMI['Instrument'] = 'HMI'
HMI['BMRLabel'] = HMI['BMRLabel'] + np.max(MDI['BMRLabel'])

In [4]:
KPVT = KPVT.loc[KPVT['Date']<np.min(SPMG['Date']),:]
SPMG = SPMG.loc[SPMG['Date']<np.min(MDI['Date']),:]
MDI = MDI.loc[MDI['Date']<np.min(HMI['Date']),:]

In [5]:
allMag = pd.concat([KPVT, SPMG, MDI, HMI], ignore_index=True).drop(labels=['ReferenceDay', 'Flux_cut'], axis=1).replace(np.nan, False)
allMag.to_csv('data/output/BARD_1976-2016_Merge_tilts.csv', index=False)

allMag['pltSize'] = (np.sqrt(allMag['BMRFlux']/1e21))
allMag

,Year,Month,Day,Hour,Minute,Second,BMRLabel,PFlux,PArea,PLatitude,...,AH_23s,Cycle23n,Cycle23s,HH_24n,AH_24n,HH_24s,AH_24s,Cycle24n,Cycle24s,pltSize
0,1976,1,13,17,26,59.0,1,6.690000e+21,1.590000e+19,-9.9094,...,False,False,False,False,False,False,False,False,False,4.614109
1,1976,2,15,18,19,8.0,3,2.370000e+21,1.170000e+19,-11.1080,...,False,False,False,False,False,False,False,False,False,1.939072
2,1976,2,20,18,43,18.0,4,2.490000e+21,1.440000e+19,-2.6919,...,False,False,False,False,False,False,False,False,False,2.209072
3,1976,3,13,21,24,36.0,8,3.080000e+21,1.270000e+19,-33.7100,...,False,False,False,False,False,False,False,False,False,2.424871
4,1976,8,12,21,14,0.0,13,1.790000e+21,9.400000e+18,23.7860,...,False,False,False,False,False,False,False,False,False,1.769181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7565,2016,6,12,21,46,46.7,16059,4.150000e+21,2.040000e+19,4.1443,...,False,False,False,True,False,False,False,True,False,2.716616
7566,2016,6,17,21,46,46.9,16063,3.950000e+21,1.390000e+19,-8.5136,...,False,False,False,False,False,True,False,False,True,2.505993
7567,2016,6,18,21,46,47.0,16066,2.080000e+21,1.060000e+19,6.2451,...,False,False,False,True,False,False,False,True,False,2.017424
7568,2016,6,20,21,46,47.0,16068,1.150000e+22,2.030000e+19,-6.1588,...,False,False,False,False,False,True,False,False,True,3.506565


In [9]:
df = pd.DataFrame(allMag.BMRLabel)
for k in allMag.BMRLabel:
    df_k = (allMag.BMRLabel==k)*1
    suma = df_k.sum()#[0]
    if suma > 1:
        print('Problema en BMRLabel: ', k, ' n de repeticiones: ', suma)
        print(allMag.loc[allMag.BMRLabel == k, ['BMRLatitude' ,'BMRLongitude', 'Year', 'Month', 'Day', 'PFlux', 'NFlux', 'BMRFlux', 'BMRArea']])

Problema en BMRLabel:  247  n de repeticiones:  2
     BMRLatitude  BMRLongitude  Year  Month  Day         PFlux         NFlux  \
135      19.8925      -105.256  1978      2   18  1.420000e+22 -1.290000e+22   
136      20.0570      -104.531  1978      2   22  1.690000e+22 -1.020000e+22   

          BMRFlux       BMRArea  
135  2.710000e+22  7.380000e+19  
136  2.710000e+22  7.055000e+19  
Problema en BMRLabel:  247  n de repeticiones:  2
     BMRLatitude  BMRLongitude  Year  Month  Day         PFlux         NFlux  \
135      19.8925      -105.256  1978      2   18  1.420000e+22 -1.290000e+22   
136      20.0570      -104.531  1978      2   22  1.690000e+22 -1.020000e+22   

          BMRFlux       BMRArea  
135  2.710000e+22  7.380000e+19  
136  2.710000e+22  7.055000e+19  
Problema en BMRLabel:  630  n de repeticiones:  2
     BMRLatitude  BMRLongitude  Year  Month  Day         PFlux         NFlux  \
303     -22.5835       -164.90  1978      9   30  5.770000e+21 -8.520000e+21   
304 